In [195]:
import pandas as pd
import os
import numpy as np

In [196]:
os.chdir('R:\AmanovRA')

In [197]:
data = pd.read_excel('Врачи.xlsx')

In [198]:
data = data.replace(r'^-$', np.NaN, regex=True)

In [199]:
data = data.dropna()

In [200]:
data = data[~data['Структура 3 уровень'].isin(['Обособленное подразделение Валдай', 'Операционный блок', 'Линия экспертизы застрахованного и прикрепленного контингента',
       'Общепроизводственные задачи', 'ЦВОП Бадаева',
       'Администраторы амбулаторной сети', 'Страховой стол',
       'Управление Разработки ПО и Бизнес-процессов'])]

data = data.rename(columns = {'Count(distinct[Номер карты])': 'Кол-во пациентов'})

data['Структура 3 уровень'] = data['Структура 3 уровень'].map({
    'Отделение амбулаторных медицинских сестёр': 'Сестры',
       'Отделение медицинских сестёр на линии': 'Сестры',
       'Отделение медицинских сестёр сестринского ухода': 'Сестры',
       'Отделение Рентгенологии': 'Рентген', 
       'Терапевтическое отделение Север': 'Терапевты',
       'Дерматологическое отделение': 'Дерматологи',
       'Оториноларингологическое направление': 'ЛОР',
       'Эндоскопическое отделение': 'Эндоскопическое', 
       'Терапевтическое отделение Юг': 'Терапевты',
       'Хирургическое направление': 'Хирурги', 
        'Сектор врачей': 'Стоматология',
       'Терапевтическое отделение Восток': 'Терапевты',
       'Терапевтическое отделение Северо-Запад': 'Терапевты',
       'Урологическое отделение': 'Урологи', 
        'Офтальмологическое направление': 'Офтальмологи',
       'Направление "Ультразвуковой и функциональной диагностики"': 'УЗИ',
       'Гинекологическое направление': 'УЗИ',
       'Направление "Амбулаторные педиатрические услуги"': 'Педиатры',
       'Направление "Травматология"': 'Травматологи',
       'Неврологическое отделение взрослое': 'Неврологи',
       'Психотерапевтическое отделение': 'Психотерапевты',
       'Направление  "Амбулаторная реабилитация"': 'Реабилитация',
       'Направление "Выездные педиатрические услуги"': 'Педиатры',
       'Неврологическое отделение детское': 'Неврологи',
       'Анестезиологическое отделение': 'Анестезиологи', 
       'Аллергологическое отделение': 'Аллергологи',
       'Эндокринологическое отделение': 'Эндокринологи',
       'Кардиологическое отделение детское': 'Кардиологи',
       'Отделение ассистентов терапевтов': 'Сестры',
       'Отделение восстановительной медицины': 'Реабилитация', 
        'Отделение массажа и ФТЛ': 'Сестры',
       'Направление  "Выездная реабилитация"': 'Реабилитация'
    
})

# Плохой баланс классов, сейчас оставлю с >= 4 процентов

In [201]:
bad_balance = list(map(str.strip, '''Травматологи       
Офтальмологи       
Стоматология       
Урологи            
Дерматологи        
Эндоскопическое    
Эндокринологи      
Кардиологи         
Рентген          
Аллергологи        
Психотерапевты     
Анестезиологи '''.split('\n')))

bad_balance

['Травматологи',
 'Офтальмологи',
 'Стоматология',
 'Урологи',
 'Дерматологи',
 'Эндоскопическое',
 'Эндокринологи',
 'Кардиологи',
 'Рентген',
 'Аллергологи',
 'Психотерапевты',
 'Анестезиологи']

In [202]:
data = data[~data['Структура 3 уровень'].isin(bad_balance)]

In [203]:
data_new = data.pivot_table(index='Исполнитель', columns='Месяц', values=['Выработка', 'Кол-во услуг', 'Кол-во пациентов'],
                           aggfunc={'Выработка': sum, 'Кол-во услуг': sum, 'Кол-во пациентов': sum})

In [204]:
data_new.columns = list(map(' '.join, data_new.columns))

In [205]:
data_new = data_new.merge(data[['Исполнитель', 'Структура 3 уровень']], left_index=True, right_on='Исполнитель')

In [206]:
data_new = data_new.drop_duplicates(subset='Исполнитель')

In [207]:
data_new = data_new.fillna(0).reset_index(drop=True)

In [208]:
X, y = data_new[[column for column in data_new.columns if column not in ('Исполнитель', 'Структура 3 уровень') ]], data_new['Структура 3 уровень']

In [209]:
from sklearn.model_selection import train_test_split

In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [211]:
from sklearn.preprocessing import StandardScaler

In [212]:
scaler = StandardScaler().fit(X_train)

In [213]:
X_train_new, X_test_new = scaler.transform(X_train), scaler.transform(X_test)

In [214]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

params = {'n_neighbors': list(range(2, 15))}

gcv = GridSearchCV(estimator=KNeighborsClassifier(), cv=3, param_grid=params)

gcv.fit(X_train_new, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [215]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, gcv.predict(X_train_new))

0.6728187919463087

In [216]:
accuracy_score(y_test, gcv.predict(X_test_new))

0.6333333333333333

In [217]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train_new, y_train)

C:\Users\AmanovRA\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [218]:
accuracy_score(y_train, lr.predict(X_train_new))

0.7114093959731543

In [219]:
accuracy_score(y_test, lr.predict(X_test_new))

0.64

In [220]:
y_train.nunique()

8

In [221]:
from sklearn.metrics import precision_score, recall_score



In [222]:
precision_score(y_train, gcv.predict(X_train_new), average='micro'), precision_score(y_test, gcv.predict(X_test_new), average='micro'), precision_score(y_train, lr.predict(X_train_new), average='micro'),precision_score(y_test, lr.predict(X_test_new), average='micro')

(0.6728187919463087, 0.6333333333333333, 0.7114093959731543, 0.64)

In [223]:
recall_score(y_train, gcv.predict(X_train_new), average='micro'), recall_score(y_test, gcv.predict(X_test_new), average='micro'), recall_score(y_train, lr.predict(X_train_new), average='micro'),recall_score(y_test, lr.predict(X_test_new), average='micro')

(0.6728187919463087, 0.6333333333333333, 0.7114093959731543, 0.64)

In [227]:
df = pd.DataFrame({'predicted': lr.predict(X_test_new), 'true_y': y_test})

In [231]:
df[df['predicted'] == df['true_y']].shape

(96, 2)